
# Ungraded Lab: CelebA GAN Experiments

This lab will demonstrate a GAN trained on the CelebA dataset. This is a resource-intensive task so you will use a TPU and a distributed strategy to train the network. It will take 40 to 50 minutes to run the entire exercise. Afterwards, you will see a gif showing new faces generated by the trained model.

https://github.com/sasidhar-programmer/Tensorflow_Advance_Techniques/blob/main/4-Generative-deeplearning-with-tensorflow/week-4/C4_W4_Lab_3_CelebA_GAN_Experiments.ipynb